# Chapter online
[Chapter 8](https://allendowney.github.io/ThinkBayes2/chap08.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as Dfs
import Distributions as Dsts
import CairoMakie as Cmk

In [ ]:
include("./empiricaldist.jl")
import .EmpiricalDistributions:
    EmpiricalDistributions as ed, pmf, cdf
include("./simplestat.jl")
import .SimpleStatistics as ss

## Code from chapter

### Functionality developed in Chapter 8

In [ ]:
function mkPoissonPmf(lam::Float64,
        qs::Vector{T})::pmf.Pmf{T} where T<:Union{Int, Float64}
    @assert lam >= 0
    ps::Vector{Float64} = Dsts.pdf.(Dsts.Poisson(lam), qs)
    pmfDist::pmf.Pmf{T} = pmf.Pmf(qs, ps)
    pmfDist.priors = pmfDist.priors ./ sum(pmfDist.priors)
    return pmfDist
end

### The World Cup Problem

In the 2018 FIFA World Cup final, France defeated Croatia 4 goals to 2. Based on this outcome:
- How confident should we be that France is the better team?
- If the same teams played again, what is the chance France would win again?

To answer these questions, we have to make some modeling decisions.
- First, I’ll assume that for any team against another team there is some unknown goal-scoring rate, measured in goals per game, which I’ll denote with the Python variable `lam` or the Greek letter $\lambda$, pronounced “lambda”.
- Second, I’ll assume that a goal is equally likely during any minute of a game. So, in a 90 minute game, the probability of scoring during any minute is $\lambda / 90$.
- Third, I’ll assume that a team never scores twice during the same minute."

### The Poisson Distribution

Assumption: the number of goals follows a [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution).
Goal-scoring rate is $\lambda$, the probability of scoring `k` goals is:

$\lambda^k exp(-\lambda) / k!$

for any non-negative value of `k`.

We can create one with $\lambda = 1.4$ like this:


In [ ]:
lam = 1.4
dist = Dsts.Poisson(lam)
k = 4
# the probability of scoring 4 goals in a game is about 4%
# when the avg goal-scoaring rate is 1.4
Dsts.pdf(dist, k) 

In [ ]:
lam = 1.4
goals = collect(0:10)
pmfGoals = mkPoissonPmf(lam, goals)

In [ ]:
fig = Cmk.Figure()
ax, bp = Cmk.barplot(fig[1, 1], pmfGoals.names, pmfGoals.priors,
        axis = (;
            title = "Distribution of goals scored",
            xlabel = "Number of goals",
            ylabel = "PMF",
            xticks = 0:10)
    )
Cmk.axislegend(
    ax,
    [bp],
    ["Poisson distribution with λ = 1.4"])
fig

### The Gamma Distribution

Assumption:
- good team plays against a bad one, it scores more goals (higher lambda)
- bad team plays against a good one, it scores less goals (higher lambda)

We model that with [gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) because:

1. The goal scoring rate is continuous and non-negative, and the gamma distribution is appropriate for this kind of quantity.
2. The gamma distribution has only one parameter, alpha, which is the mean. So it’s easy to construct a gamma distribution with the mean we want.
3. As we’ll see, the shape of the gamma distribution is a reasonable choice, given what we know about soccer.

In [ ]:
alpha = 1.4
qs = range(0, 10, 101)
ps = Dsts.pdf.(Dsts.Gamma(alpha), qs);

In [ ]:
pmfGoalsGamma = pmf.Pmf(collect(qs), ps)
pmf.calculatePosteriors!(pmfGoalsGamma);

In [ ]:
fig = Cmk.Figure()
ax, lp = Cmk.lines(fig[1, 1],
    pmfGoalsGamma.names, pmfGoalsGamma.posteriors,
        axis = (;
            title = "Prior distribution of λ",
            xlabel = "Goal scoring rate (lam)",
            ylabel = "PMF",
            xticks = 0:10)
    )
Cmk.axislegend(
    ax,
    [lp],
    ["prior value for lam"])
fig

In [ ]:
# confirmation that the mean is about 1.4
pmf.getMean(pmfGoalsGamma)